<center><h2><font color="red" size="+3">Reading Aeronet Data Files with Pandas</font></h2></center>

In [ ]:
import warnings
warnings.filterwarnings("ignore") # Suppress warning messages

In [ ]:
import datetime
import numpy as np
import pandas as pd
print('Using pandas version ',pd.__version__)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
# Use seaborn style defaults and set the default figure size
sns.set(rc={'figure.figsize':(11, 4)})

### Reading the data

- The file contains 6 metadata rows (to be skipped).
- Values are separated by commas (`,`).

In [ ]:
url = "https://raw.githubusercontent.com/astg606/py_materials/master/aeronet/"
filename = "19930101_20190209_Dakar.ONEILL_lev20"

#filename = "19930101_20190209_GSFC.lev20"

In [ ]:
aero_df = pd.read_csv(url+filename, skiprows=6, sep=",")

In [ ]:
aero_df

Delete last column:

In [ ]:
aero_df.columns[len(aero_df.columns)-1]

In [ ]:
aero_df.drop(aero_df.columns[len(aero_df.columns)-1], axis=1, inplace=True)

In [ ]:
aero_df

In [ ]:
aero_df.info

In [ ]:
aero_df.describe().transpose()

#### Combine the first two columns into a `datetime` object

- We combine the first two columns (Date and Time) into one unique `Datetime` object that will now be the column of indices. This will allows us to easily manipualte the data.
- Set all `-999` as missing values (`NaN`).

In [ ]:
dateparse = lambda x: datetime.datetime.strptime(x, '%d:%m:%Y %H:%M:%S')
aero_df = pd.read_csv(filename, skiprows=6, na_values=-999,
                      parse_dates={'datetime': [0, 1]}, 
                      date_parser=dateparse, index_col=0, squeeze=True)

In [ ]:
aero_df

#### Drop any rows that are all `NaN` and any cols that are all `NaN`

In [ ]:
aero_df = aero_df.dropna(axis=1, how='all').dropna(axis=0, how='all')

In [ ]:
aero_df

#### Select four columns to form a new DataFrame

In [ ]:
list(aero_df.columns)

In [ ]:
new_df = aero_df[['Total_AOD_500nm[tau_a]',
                  'Fine_Mode_AOD_500nm[tau_f]',
                  'Coarse_Mode_AOD_500nm[tau_c]',
                  'FineModeFraction_500nm[eta]']]

In [ ]:
new_df

Drop column or row with all `NaN`

In [ ]:
new_df = new_df.dropna(axis=1, how='all').dropna(axis=0, how='all')

Rename the column:

In [ ]:
new_df.columns = ['aot', 'aotc', 'faotf', 'faotc']

In [ ]:
new_df.describe().transpose()

#### Determine the daily means:

In [ ]:
new_df_dm = new_df.resample("D").mean()

In [ ]:
new_df_dm

In [ ]:
new_df_dm = new_df_dm.dropna(axis=1, how='all').dropna(axis=0, how='all')

In [ ]:
new_df_dm

Perform various plots

In [ ]:
new_df_dm.plot(linewidth=0.5, subplots=True);

In [ ]:
new_df_dm['2012':'2018'].plot(linewidth=0.9, subplots=True);

In [ ]:
new_df_dm['2012':'2018'].plot.area(linewidth=0, subplots=True);

In [ ]:
import matplotlib.dates as mdates

df = new_df_dm['2012':'2018']

fig, ax = plt.subplots(nrows=4, ncols=1, sharex=True)

ax[0].plot(df.aot, marker='.', 
           markersize=2, color='0.6', linestyle='None', label='aot')
#ax[0].xaxis.set_major_locator(mdates.YearLocator())
ax[0].legend()
ax[0].set_xlabel('Year')

ax[1].plot(df.aotc, marker='.', 
           markersize=2, color='0.6', linestyle='None', label='aotc')
#ax[1].xaxis.set_major_locator(mdates.YearLocator())
ax[1].legend()
ax[1].set_xlabel('Year')

ax[2].plot(df.faotf, marker='.', 
           markersize=2, color='0.6', linestyle='None', label='faotf')
#ax[2].xaxis.set_major_locator(mdates.YearLocator())
ax[2].legend()
ax[2].set_xlabel('Year')

ax[3].plot(df.faotc, marker='.', 
           markersize=2, color='0.6', linestyle='None', label='faotc')
#ax[3].xaxis.set_major_locator(mdates.YearLocator())
ax[3].legend()
ax[3].set_xlabel('Year')

plt.tight_layout()

#### Determine the annual means:

In [ ]:
new_df_am = new_df.resample("A").mean()

In [ ]:
new_df_am

In [ ]:
new_df_am.plot(style='b--', subplots=True);

#### Extract the values for 2017 only

In [ ]:
df2017 = new_df[new_df.index.year == 2017]

In [ ]:
df2017

In [ ]:
df2017.plot(style='b--', subplots=True);

#### Extract the values for April 2017 only

In [ ]:
df201704 = df2017[df2017.index.month == 4]

In [ ]:
#df201704 = new_df[new_df.index.year == 2017 & new_df.index.month == 4]

In [ ]:
df201704

In [ ]:
df201704.plot(kind='scatter', x='aot', y='aotc');

#### Average daily values of April 2017

In [ ]:
df201704_dm = df201704.resample("D").mean()

In [ ]:
df201704_dm

In [ ]:
df201704_dm.describe().transpose()

# GSFC - Beijing - Kanpur

In [ ]:
fname_Beijing = url+'19930101_20210102_Beijing.lev20'
fname_GSFC = url+'19930101_20210102_GSFC.lev20'
fname_Kanpur = url+'19930101_20210102_Kanpur.lev20'

In [ ]:
dateparse = lambda x: datetime.datetime.strptime(x, '%d:%m:%Y %H:%M:%S')
df = pd.read_csv(fname_Beijing, skiprows=6, na_values=-999,
                      parse_dates={'datetime': [0, 1]}, 
                      date_parser=dateparse, index_col=0, squeeze=True)

In [ ]:
df.head()

In [ ]:
list(df)

In [ ]:
old_cols = ['Day_of_Year', 'AOD_675nm', 'AOD_440nm', 
            '440-675_Angstrom_Exponent', 
             'Site_Latitude(Degrees)', 'Site_Longitude(Degrees)',
             'Site_Elevation(m)']

new_cols = ['DoY', 'A675', 'A440', 'Alpha', 'Lat', 'Lon', 'Alt']

In [ ]:
df_Beijing = df[old_cols]

In [ ]:
df_Beijing.columns = new_cols
df_Beijing.head()

In [ ]:
df_Beijing['A550'] = df_Beijing['A675']*((675.0/550.0))**df_Beijing['Alpha']
df_Beijing.head()

In [ ]:
site_B = "Beijing"
lat_B = df_Beijing['Lat'][0]
lon_B = df_Beijing['Lon'][0]

print(lat_B, lon_B)

In [ ]:
df_Beijing[df_Beijing.index.year == 2010].plot(x='DoY', 
                                               y='A550', 
                                               color='k',
                                               title=site)

In [ ]:
dateparse = lambda x: datetime.datetime.strptime(x, '%d:%m:%Y %H:%M:%S')
df = pd.read_csv(fname_GSFC, skiprows=6, na_values=-999,
                      parse_dates={'datetime': [0, 1]}, 
                      date_parser=dateparse, index_col=0, squeeze=True)

In [ ]:
df_GSFC = df[old_cols]
df_GSFC.columns = new_cols
df_GSFC['A550'] = df_GSFC['A675']*((675.0/550.0))**df_GSFC['Alpha']

In [ ]:
df_GSFC['A550'].plot()

In [ ]:
site_G = "GSFC"
lat_G = df_GSFC['Lat'][0]
lon_G = df_GSFC['Lon'][0]

In [ ]:
dateparse = lambda x: datetime.datetime.strptime(x, '%d:%m:%Y %H:%M:%S')
df = pd.read_csv(fname_Kanpur, skiprows=6, na_values=-999,
                      parse_dates={'datetime': [0, 1]}, 
                      date_parser=dateparse, index_col=0, squeeze=True)

In [ ]:
df_Kanpur = df[old_cols]
df_Kanpur.columns = new_cols
df_Kanpur['A550'] = df_Kanpur['A675']*((675.0/550.0))**df_Kanpur['Alpha']

In [ ]:
site_K = "Kanpur"
lat_K = df_Kanpur['Lat'][0]
lon_K = df_Kanpur['Lon'][0]

In [ ]:
df_Kanpur['A550'].plot()

In [ ]:
df_B2010 = df_Beijing[df_Beijing.index.year == 2010]
df_K2010 = df_Kanpur[df_Kanpur.index.year == 2010]
df_G2010 = df_GSFC[df_GSFC.index.year == 2010]

In [ ]:
fig, (axG, axB, axK) = plt.subplots(nrows=3, ncols=1, figsize=(12,8))

axB.plot(df_B2010["DoY"], df_B2010['A550'], label='AERONET')
axB.set_xlabel("Days in 2010")
axB.set_title(site_B)
axB.legend()

axK.plot(df_K2010["DoY"], df_K2010['A550'], label='AERONET')
axK.set_xlabel("Days in 2010")
axK.set_title(site_K)
axK.legend()

axG.plot(df_G2010["DoY"], df_G2010['A550'], label='AERONET')
axG.set_xlabel("Days in 2010")
axG.set_title(site_G)
axG.legend()

plt.tight_layout()